In [1]:
import findspark
findspark.init()

import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_utc_timestamp, from_unixtime, to_date, sum,count, mean, countDistinct, max,first
from pyspark import SparkContext
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.types import IntegerType,StructField,StructType, LongType, FloatType
from pyspark.sql.functions import col, when

from pyspark.sql.functions import monotonically_increasing_id, coalesce, date_trunc, regexp_extract
import snap
from os import path
import glob

import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from bitcoin_utils import bitcoin_utils
import plotly
import plotly.express as px
pd.options.plotting.backend = "plotly"
import plotly.io as pio



In [2]:
spark = SparkSession \
    .builder \
    .config("spark.driver.memory", "100g") \
    .config("spark.executor.memory", "100g") \
    .config("spark.cores.max", "10") \
    .getOrCreate()
sc = spark.sparkContext

In [3]:
data= spark.read.load("/media/ssd2/bitcoinRemy/tr_everything_by_day/")

In [8]:
data

DataFrame[hash: string, output_id: bigint, src: string, dst: string, value: bigint, time: string, nb_inputs: bigint, nb_outputs: bigint, fee: bigint, src_identity: string, dst_identity: string, PriceUSD: float, valueUSD: double, spent_timestamp: string, next_nb_inputs: bigint, dst_2: string, src_2: string, dst_cat: string, src_cat: string, year: int, month: int, day: int]

In [21]:
filtered = data.filter(col("year")==2015).filter((col("month")==1)).filter("src_identity is not null").filter("dst_identity is not null").select(["src_identity","dst_identity","time","value"])

In [22]:
pd_out = filtered.select("*").toPandas()

In [25]:
only_known=pd_out[(~pd_out["src_identity"].astype(str).str.isnumeric())]

In [26]:
only_known=only_known[(~pd_out["dst_identity"].astype(str).str.isnumeric())]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [32]:
only_known = only_known[only_known["src_identity"]!=only_known["dst_identity"]]

In [ ]:
only_known = only_known.sort_values("time")

In [34]:
only_known.sort_values("time").to_csv("/media/ed5/remyBitcoin/2015-01-known.csv")